공공데이터 활용하여 신입생 수 받아오기
===

- 프로그램 제작 목적
    1. 입찰데이터를 관리하려면 매년 신입생 수를 시스템에 기입해야 한다.
    1. 학교알리미라는 정부사이트에서는 매년 전국학교의 신입생 수를 제공한다.
    1. 영업팀에서는 직접 홈페이지를 방문해서 수치를 확인한다.
    1. 공공데이터 포탈에는 있는 학교알리미 사이트의 신입생 수 정보 API 가 있다.
    1. 정리된 엑셀파일로 영업팀에 신입생 수를 제공하여 업무 효율을 높이자.
    
- 프로그램에 자동기입까지 전부 자동화하는 것에 대해
    1. 제작은 가능하나 전제조건이 성립되지 않아 중단했다.
    2. 전제조건
        1. 기본적 테이블은 모두 준비되어 있는 상태다.
        1. 아이비클럽 실학교 코드의 명칭을 학교알리미의 정식학교 명칭과 통일해야한다.
        1. 각 상권담당자들에게 기입을 맡겨놓아 기입룰에 맞지 않는 명칭이 너무 많다.
        1. 실 데이터를 전산팀에서 교정하며 진행할 수 있는 분량은 아니고 영업팀에서 의지가 있으면 진행가능하다.

In [1]:
import requests
import pandas as pd
import sys

In [2]:
# http://www.schoolinfo.go.kr/openApi.do?apiType=09&pbanYr=2021&schulKndCode=03&apiKey=API_KEY
 
#요청url 잘게 자르기
url = "http://www.schoolinfo.go.kr/openApi.do"

# queryParams = '?' + urlencode({ quote_plus('apiType') : '63', # API 종류 (성별 학생수)
#                                 quote_plus('pbanYr') : '2021', # 공시연도 (최근3년간의 정보만 제공)
#                                 quote_plus('schulKndCode') : '03', # 학교급구분 (02:초등 03:중등 04:고등 05:특수 06:그외)
#                                 quote_plus('apiKey') : 'API_KEY'
#                                 })
# get_data = requests.get(url + unquote(queryParams), verify=False) # verify=False 없으면 ssl 인증에러남

apiType = "?apiType=63" # API 종류 (성별 학생수)
# pbanYr = "&pbanYr=2022" # 공시연도 (최근3년간의 정보만 제공)
pbanYr = "&pbanYr="
# schulKndCode = "&schulKndCode=03" # 학교급구분 (02:초등 03:중등 04:고등 05:특수 06:그외)
schulKndCode = "&schulKndCode="

# 키 가져오기
sys.path.append('/settings')
import config
apiKey = config.API_Keys['school_info_Key']

## 학생수 받아오는 함수

In [3]:
# 학생수 받아오기
def get_students(year, sch_kind):
    year = pbanYr + year
    sch_kind = schulKndCode + sch_kind
    
    # URL 및 파라메터 조립, 인증 안함
    get_data = requests.get(url + apiType + year + sch_kind + apiKey, verify=False) # verify=False 없으면 ssl 인증에러남
    
    result_data = get_data.json()
    
    df = pd.json_normalize(result_data['list'])
    
    df['년도'] = year[-4:]
    
    # 원하는 열만 뽑아내기
    df = df[['년도',
             'ADRCD_NM', # 지역
             'SCHUL_NM', # 학교명
             'COL_M1', # 1학년(남)
             'COL_W1' # 1학년(여)
            ]]
    
    # 열이름 한글변경
    df.columns = ['년도', '지역', '학교명', '1학년(남)', '1학년(여)']
    
    return df

In [4]:
df_mid_grade1 = get_students('2022', '03') # 중1
df_high_grade1 = get_students('2022', '04') # 고1

# 경고는 verify=False 때문이다.
df_mid_grade1

C:\ProgramData\Miniconda3\envs\py3.8.13\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.schoolinfo.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Miniconda3\envs\py3.8.13\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.schoolinfo.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,년도,지역,학교명,1학년(남),1학년(여)
0,2022,서울특별시 성북구,서울대학교사범대학부설중학교,185,0
1,2022,서울특별시 종로구,서울대학교사범대학부설여자중학교,0,99
2,2022,서울특별시 강남구,개원중학교,87,162
3,2022,서울특별시 서초구,경원중학교,177,145
4,2022,서울특별시 강남구,구룡중학교,118,95
...,...,...,...,...,...
3290,2022,제주특별자치도 제주시,한라중학교,144,211
3291,2022,제주특별자치도 제주시,노형중학교,142,233
3292,2022,제주특별자치도 제주시,오름중학교,141,142
3293,2022,제주특별자치도 제주시,탐라중학교,142,117


## 자료 작성기준일 입력

In [5]:
from datetime import datetime

In [6]:
today = datetime.today().strftime("%Y%m%d")
today[:8]

'20220719'

In [7]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'학교알리미_{today[:8]}_중고교_학생수.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_mid_grade1.to_excel(writer, sheet_name='중학교', index=False)
df_high_grade1.to_excel(writer, sheet_name='고등학교', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()